# 1 - Introduction

Wine quality assessment is a crucial aspect of the wine industry, influencing production, pricing, and consumer satisfaction. This dataset, sourced from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/186/wine+quality), contains physicochemical properties of red and white wine samples from the **Vinho Verde** region of Portugal. Each sample is labeled with a **quality score** ranging from 0 to 10, based on sensory evaluations from wine experts.

The objective of this study is to develop a machine learning model capable of predicting **wine quality** based on its **chemical composition**. By analyzing factors such as **acidity, alcohol content, pH, and residual sugar**, we aim to uncover patterns that distinguish high-quality wines from lower-quality ones.

This project will involve **data preprocessing, exploratory data analysis (EDA), feature engineering, and model selection** to build a predictive system that can assist winemakers in optimizing production processes and maintaining high-quality standards.

## 1.1 - Explanation of Variables  

The dataset consists of **physicochemical properties** of wine samples and a **quality score** assigned by wine experts. Below is a description of each variable:  

### **Features (Independent Variables)**  
These continuous variables represent the chemical composition of the wine:  

- **fixed_acidity**: The amount of non-volatile acids present in the wine, primarily **tartaric acid**. It contributes to the wine’s tartness.  
- **volatile_acidity**: The concentration of volatile acids, mainly **acetic acid**, which can give wine an unpleasant vinegar taste if too high.  
- **citric_acid**: A naturally occurring acid in grapes that can add freshness and enhance the wine’s **flavor profile**.  
- **residual_sugar**: The remaining **sugar** after fermentation, measured in grams per liter. Affects the wine’s **sweetness**.  
- **chlorides**: The amount of **salt** in the wine, which can influence taste and preservation.  
- **free_sulfur_dioxide**: The **unbound SO₂** that acts as an antimicrobial and antioxidant in wine.  
- **total_sulfur_dioxide**: The total amount of **free and bound SO₂**, used as a preservative to prevent oxidation and bacterial growth.  
- **density**: The **mass per unit volume** of the wine, influenced by **alcohol, sugar, and dissolved substances**.  
- **pH**: Measures the **acidity** of the wine on a scale from **0 to 14**, where lower values indicate higher acidity.  
- **sulphates**: A form of **sulfur** that contributes to **wine stability** and acts as an antimicrobial agent.  
- **alcohol**: The **ethanol content** of the wine, which significantly impacts its **body, taste, and perception of quality**.  

### **Target Variable (Dependent Variable)**  
- **quality**: The **expert-assigned score** for each wine, ranging from **0 to 10**. A higher score indicates better wine quality.  

### **Other Variable**  
- **color**: A **categorical variable** indicating whether the wine is **red or white**.  

Each of these variables plays a role in defining wine characteristics, and understanding their interactions can help build a robust predictive model for wine quality.


In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style('darkgrid')  # Set the plot style
palette = sns.color_palette(palette="pastel")
sns.set_palette("pastel")

import matplotlib.pyplot as plt

from scipy.stats import ttest_ind, kruskal, rankdata, norm
import scipy.stats as stats
from scipy.spatial.distance import pdist, squareform

#Model
from sklearn.model_selection import train_test_split

# 2.1 Preparing the data

In [2]:
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 

ConnectionError: Error connecting to server

In [ ]:
# variable information 
print(wine_quality.variables) 

In [ ]:
df_wine = wine_quality.data.original

In [ ]:
df_wine.head()

## 2.2 Checking for data quality

In [ ]:
#Null values
df_wine.isna().sum()

In [ ]:
#Duplicated values
duplicated = df_wine[df_wine.duplicated(keep = False)]
print(duplicated.shape)

In [ ]:
duplicated.head(15)

In [ ]:
df_wine_clean_all = df_wine.drop_duplicates(keep = 'first')
print(df_wine_clean_all.shape)

## Data Splitting for Training, Validation, and Testing

To ensure that our model is trained and evaluated properly, it's crucial to split the dataset into three distinct subsets: **training**, **validation**, and **test**. This is important to prevent **data leakage**, which can occur if the model has access to the test data during the training process, leading to overly optimistic performance estimates.

For our analysis, we will use the following split:
- **70% of the data** for training the model
- **20% of the data** for validating the model during training (to tune hyperparameters)
- **10% of the data** for final testing, ensuring an unbiased evaluation of the model's performance after training.

By keeping these datasets separate, we can avoid bias and ensure a more robust evaluation of the model's generalizability to new, unseen data.

In [ ]:
# Separate the features (X) and target variable (y)
X = df_wine_clean_all.drop('quality', axis=1)  # Assuming 'quality' is the target
y = df_wine_clean_all['quality']

# First, split the data into training (80%) and temporary data (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Then, split the temporary data into validation (14%) and test (6.66%)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)

# Display the sizes of the resulting sets
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_valid.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
df_wine_clean = pd.concat([X_train.copy(), y_train], axis=1).reset_index(drop = True)

In [ ]:
df_wine_clean.head()

## 2.3 Data distribution

In [ ]:
df_wine_clean.describe()

In the next figure, we will check the data distribution and we will do a hypothesis test to see if the data is normally distributed with the Shapiri-Wilk test. The **Shapiro-Wilk test** is a statistical test used to assess whether a given sample comes from a normally distributed population. It is widely used for testing the normality of data, especially when dealing with small to moderate sample sizes.

#### Hypotheses: 
  $$ H_0: \text{The data is normally distributed} $$
  $$ H_1: \text{The data is not normally distributed} $$

In this analysis, we will use the Shapiro-Wilk test to evaluate if any of our numerical variables follow a normal distribution. If the p-value is greater than 0.05, we will assume the distribution is normal; otherwise, we will assume the distribution deviates from


In [ ]:
def plot_hist_box(df, col, style="darkgrid"):
    """
    Plots a histogram and a boxplot for a given column, sharing the x-axis, 
    and annotates the Shapiro-Wilk test result for normality.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing the data.
        col (str): Column name for the histogram and boxplot.
        style (str): Seaborn style (default: "darkgrid").
    """
    # Set the plot style
    sns.set_style(style)  
    
    # Perform the Shapiro-Wilk test for normality
    stat, p_value = stats.shapiro(df[col])
    normality_test_result = "Normally Distributed" if p_value > 0.05 else "Not Normally Distributed"
    
    # Calculate mean and median
    mean_val = df[col].mean()
    median_val = df[col].median()

    # Create figure with two subplots
    fig, axes = plt.subplots(2, 1, figsize=(8, 4), sharex=True, gridspec_kw={'height_ratios': [1, 5]})

    # Boxplot with mean marker
    sns.boxplot(data=df, x=col, ax=axes[0], orient="h", showmeans=True,
                meanprops={"marker": "o", "markerfacecolor": 'blue', "markeredgecolor": "black", "markersize": 7})

    # Histogram with mean and median lines
    sns.histplot(df[col], kde=True, ax=axes[1], color='skyblue')
    axes[1].axvline(mean_val, color='red', linestyle="--", linewidth=2, label="Mean")
    axes[1].axvline(median_val, color='green', linestyle="-.", linewidth=2, label="Median")

    # Annotate the Shapiro-Wilk test result
    axes[1].annotate(f"Shapiro-Wilk p-value: {p_value:.3e}\n{normality_test_result}",
                     xy=(0.7, 0.85), xycoords="axes fraction", fontsize=12, color="black", weight="bold", 
                     horizontalalignment='center', verticalalignment='center')

    # General title
    fig.suptitle(f"Distribution of {col}", fontsize=14, fontweight="bold")

    # Shared legend with colors matching the plot
    handles = [
        plt.Line2D([0], [0], color='red', linestyle="--", linewidth=2, label="Mean"),
        plt.Line2D([0], [0], color='green', linestyle="-.", linewidth=2, label="Median")
    ]
    fig.legend(handles=handles, loc="upper right", fontsize=10)

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [ ]:
for col in df_wine_clean.select_dtypes(include=['number']).columns:
    plot_hist_box(df_wine, col)

In [ ]:
#sns.set_style('darkgrid')

# Get value counts
counts = df_wine_clean['color'].value_counts()

# Get Seaborn colors
colors = sns.color_palette("pastel", len(counts))

# Create pie chart
fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(counts, labels=counts.index, autopct="%1.1f%%", colors=colors, startangle=90, wedgeprops={"edgecolor": "black"})

# Title
plt.title(f"Distribution of Wine Color", fontsize=14, fontweight="bold")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(data=df_wine_clean, x='quality', hue='color', kde=True, stat="percent", discrete=True, 
             common_norm=False, multiple="layer", palette="pastel", ax=ax)

# Labels and title
ax.set_xlabel("Wine Quality", fontsize=12, fontweight="bold")
ax.set_ylabel("Proportion Density", fontsize=12, fontweight="bold")
ax.set_title("Wine Quality Distribution by Color", fontsize=14, fontweight="bold")
plt.show()

In [ ]:
df_wine_clean[['quality','color']].groupby(['quality','color']).size().reset_index()

## 2.4 Outliers

In [ ]:
# Lets reset the index
df_wine_clean.reset_index(inplace = True, drop = True) 

In [ ]:
def index_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outlier_index = df[(df[column]< lower_bound) | (df[column]> upper_bound)].index
    print(f"Number of outliers for column {column}: {len(outlier_index)} ({len(outlier_index)/len(df)*100:.2f}%)")

    # Create the cumulative histplot
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column], kde=False, cumulative=True, color='skyblue', stat="density", bins=30)
    
    # Add horizontal lines for the 95% and 99% percentiles
    percentile_95 = df[column].quantile(0.95)
    percentile_99 = df[column].quantile(0.99)
    plt.axvline(percentile_95, color='green', linestyle='--', label=f'95% percentile ({percentile_95:.2f})')
    plt.axvline(percentile_99, color='red', linestyle='--', label=f'99% percentile ({percentile_99:.2f})')
    
    # Add lines for the outlier bounds
    plt.axvline(lower_bound, color='orange', linestyle='--', label=f'Lower bound ({lower_bound:.2f})')
    plt.axvline(upper_bound, color='orange', linestyle='--', label=f'Upper bound ({upper_bound:.2f})')

    # Labels and title
    plt.title(f'Cumulative Histogram of {column} with Outlier Bounds')
    plt.xlabel(column)
    plt.ylabel('Cumulative Density')
    
    # Add legend
    plt.legend()
    
    # Show plot
    plt.show()
    
    return outlier_index

In [ ]:
outliers_index = {}
for col in df_wine_clean.select_dtypes(include=['number']).columns:
    outliers_index[col] = index_outliers(df_wine_clean, col)

### **Highlights:**  
- The vast majority of wines have a quality rating between **5 and 7**.  
- **White wines** tend to have **higher quality ratings** compared to red wines.  
- The dataset consists of **75% white wines and 25% red wines**.  
- Very few wines have **extremely high or extremely low quality ratings**.  
- **Boxplots and cumulative histograms suggest the presence of outliers** in many variables, though the values do not seem unreasonable. **We need to evaluate the impact on the model performance later**
- The dataset originally contained **2,139 duplicate rows**, which were removed.  
- **No missing values** were found.  

### **Next Steps:**    
- Investigate how the wine color impacts other variables. Is it worth separating?  
- Analyze the relationships between variables (bivariate analysis).  
- Identify which variables have the strongest correlation with the target.

# 3 - Exploratory data analysis

In [ ]:
# Create quality category

df_wine_clean['quality_categorical'] = df_wine_clean['quality'].apply(lambda x: 'Good' if x>=7 else ('Medium' if x>=5 else 'Bad'))


quality_order = ["Bad", "Medium", "Good"]

# Convert to an ordered categorical variable
df_wine_clean["quality_categorical"] = pd.Categorical(
    df_wine_clean["quality_categorical"], 
    categories=quality_order, 
    ordered=True
)

In [ ]:
numerical_columns = df_wine_clean.select_dtypes(include=['number']).columns
categorical_columns = df_wine_clean.select_dtypes(exclude=['number']).columns

In [ ]:
df_wine_clean['quality_categorical'].value_counts()

## 3.1 - Color of wine

### 3.1.2 - Wine color impacts on quality categorical

In [ ]:
# Define the order for wine quality categories
quality_order = ['Good', 'Medium', 'Bad']

plt.figure(figsize=(10, 6))
ax = sns.histplot(data=df_wine_clean, x='color', hue='quality_categorical', multiple='stack', discrete=True, 
                  hue_order=quality_order, shrink=0.8)

# Add title and labels
plt.title('Wine Quality Distribution by Wine Color', fontsize=14)
plt.xlabel('Wine Color', fontsize=12)
plt.ylabel('Count', fontsize=12)

# Hide y-axis labels
ax.set_yticklabels([])

# Dictionary to store cumulative heights per category per x-value
cumulative_heights = {}

# Iterate through patches to annotate percentages
for patch in ax.patches:
    height = patch.get_height()
    if height > 0:
        # Get the x-value (wine color)
        x_value = patch.get_x() + patch.get_width() / 2.
        
        # Get the current cumulative height for this x_value
        bottom_height = cumulative_heights.get(x_value, 0)
        
        # Compute the middle height for annotation
        middle_height = bottom_height + height / 2.
        
        # Calculate total count for this x_value (to get the percentage)
        total = sum([p.get_height() for p in ax.patches if p.get_x() + p.get_width() / 2. == x_value])
        percentage = (height / total) * 100
        
        # Annotate inside the correct bar section
        ax.annotate(f'{percentage:.1f}%', 
                    (x_value, middle_height), 
                    ha='center', va='center', fontsize=10, color='black', weight='bold')
        
        # Update cumulative height for this x_value
        cumulative_heights[x_value] = bottom_height + height

# Show the plot with adjusted layout
plt.tight_layout()
plt.show()

In [ ]:
#Evaluate chi-square value
contingency_table = pd.crosstab(df_wine_clean['color'], df_wine_clean['quality_categorical'])

# Perform chi-square test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi-Square Statistic: {chi2:.4f}")
print(f"P-value: {p:.3e}")

# Interpret results
alpha = 0.05  # Significance level
if p < alpha:
    print("There is a significant difference in wine quality distribution between red and white wines.")
else:
    print("No significant difference in wine quality distribution between red and white wines.")

### 3.1.2 - Wine Color Impacts on Numerical Columns  

In this section, we analyze how wine color (red or white) influences the distribution of numerical variables. Since red and white wines are produced using different grape varieties and fermentation processes, we expect significant differences across various attributes.  

To statistically evaluate these differences, we perform a **t-test for independent samples**, assuming unequal variance. The hypotheses tested are:  

> **$H_0: \ \mu_1 - \mu_2 = 0$: There is no difference in means between red and white wines**  
> **$H_1: \ \mu_1 - \mu_2 \neq 0$ The means of red and white wines are significantly different**  

In the following plots, we visualize the distributions of numerical variables by wine color using boxplots. Additionally, we annotate each plot with the **p-value** from the t-test to highlight the statistical significance of the observed differences.  

In [ ]:
fig, axes = plt.subplots(nrows=(len(numerical_columns) + 1) // 2, ncols=2, figsize=(14, len(numerical_columns)*2))
axes = axes.flatten()  # Flatten to make it easier to iterate over axes

# Loop through the columns to plot each boxplot
for i, col in enumerate(numerical_columns):
    # Create the boxplot
    sns.boxplot(data=df_wine_clean, x='color', y=col, ax=axes[i], orient="v", showmeans=True,
                meanprops={"marker": "o", "markerfacecolor": palette[3], "markeredgecolor": "black", "markersize": 7})
    
    # Perform the t-test for the two categories of wine (assuming 'color' is the column with wine types)
    group1 = df_wine_clean[df_wine_clean['color'] == 'red'][col]
    group2 = df_wine_clean[df_wine_clean['color'] == 'white'][col]
    
    t_stat, p_value = ttest_ind(group1, group2, equal_var = False, alternative = 'two-sided')
    
    # Add annotation with t-statistic and p-value
    axes[i].annotate(f"T-statistic: {t_stat:.2f}\nP-value: {p_value:.3e}", 
                     xy=(0.5, 0.9), xycoords='axes fraction', ha='center', va='top', 
                     fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black', linewidth=1))

    # Set title and labels
    axes[i].set_title(f'{col}', fontsize=14)
    axes[i].set_xlabel('')
    axes[i].set_ylabel(col, fontsize=12)

# Remove empty subplots if the number of variables is odd
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

- All metrics showed a significant difference between white and red wines, which is natural given that they use different grapes and processes.  
- We used the t-test for evaluation, considering variance differences (due to the wine color distinction) and a two-tailed test.  
- Color appears to be a distinguishing characteristic.  


## 3.2 Wine quality separability

In [ ]:
# Compute correlation matrix
corr_matrix = df_wine_clean[numerical_columns].corr()

# Mask the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    corr_matrix, 
    mask=mask,   # Apply mask
    annot=True,  # Show correlation values
    fmt=".2f",   # Two decimal places
    cmap="coolwarm", 
    linewidths=0.5, 
    cbar=True
)

# Title
plt.title("Correlation Heatmap of Numerical Features", fontsize=14)
plt.show()

In [ ]:
g = sns.pairplot(data=df_wine_clean.drop('quality', axis=1), hue='quality_categorical')

plt.show()

In [ ]:
n_cols = 2
n_rows = int(np.ceil(len(numerical_columns) / n_cols))

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4), sharex=True)
axes = axes.flatten()  # Flatten for easier indexing

# Loop through numerical columns and plot each
for i, col in enumerate(numerical_columns):
    sns.boxplot(
        data=df_wine_clean, 
        y=col, 
        x="quality_categorical", 
        ax=axes[i], 
        orient="v",  # Vertical orientation
        showmeans=True,
        meanprops={"marker": "o", "markerfacecolor": palette[3], "markeredgecolor": "black", "markersize": 7}
    )
    axes[i].set_title(f"{col}", fontsize=14)
    axes[i].set_ylabel(col, fontsize = 14)
    axes[i].set_xlabel("", fontsize = 14)
    

    # Apply global X-axis limits

# Hide empty subplots (if any)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()

### Hypothesis Testing for Quality categorical effects using Kruskal-Wallis Test

Since the data is not normally distributed (from previous analysis), we'll use the **Kruskal-Wallis Test**, which is a non-parametric test that compares the medians of three or more independent groups. It does not assume a normal distribution and is appropriate for ordinal categorical variables like "Bad", "Medium", and "Good" in this case.

#### Formulation of Hypothesis:

Let’s consider the variable **X** as a numerical feature (e.g., alcohol content, pH, etc.) and **Y** as the ordinal categorical variable (with levels "Bad", "Medium", and "Good"). The hypothesis for this test are:

> $H_0: \mu_1 = \mu_2 = \mu_3$

> $H_1: \text{At least one of the medians is different}$

In [ ]:
for col in numerical_columns:
    print(f"Testing column: {col}")
    
    # Group data by quality_categorical and collect the corresponding column values
    groups = [df_wine_clean[df_wine_clean['quality_categorical'] == quality][col] 
              for quality in df_wine_clean['quality_categorical'].unique()]
    
    # Perform Kruskal-Wallis Test
    stat, p_value = kruskal(*groups)
    
    # Print the result
    print(f"Kruskal-Wallis test statistic: {stat:.3e}")
    print(f"P-value: {p_value:.3e}")
    
    # Determine significance
    if p_value < 0.05:
        print("The difference is statistically significant.")
    else:
        print("No statistically significant difference.")
    print("-" * 50)

### Post-Hoc Test to Determine Which Groups Differ

After performing the Kruskal-Wallis test and finding significant differences in the medians across the groups, we proceed with a **Post-Hoc Test** to identify which specific pairs of groups are different. To achieve this, we use **Dunn's Test** with a **Bonferroni correction** to account for multiple comparisons.

#### Dunn's Test:

- Dunn's test is a non-parametric test that compares all pairs of groups. Since the Kruskal-Wallis test indicated significant differences, Dunn's test helps pinpoint which specific pairs of groups (e.g., "Bad" vs. "Medium", "Medium" vs. "Good", etc.) have statistically significant differences in their distributions.

#### Bonferroni Correction:

- As we are making multiple comparisons, a **Bonferroni correction** is applied to control for Type I errors. This is done by adjusting the significance level (0.05) based on the number of comparisons made.

In [ ]:
def dunn_test(data, groups):
    """
    Perform Dunn's test for pairwise comparisons of groups
    with a Bonferroni correction for multiple testing.
    
    Args:
        data: List of arrays for each group (e.g., values for "Bad", "Medium", "Good")
        groups: List of group names (e.g., ['Bad', 'Medium', 'Good'])
    
    Returns:
        adjusted_p_values: List of tuples (group1, group2, adjusted p-value)
    """
    p_values = []
    # Loop over all unique pairs of groups
    for i, group1 in enumerate(groups):
        for j, group2 in enumerate(groups):
            if i < j:
                group1_data = np.array(data[i])
                group2_data = np.array(data[j])
                n1 = len(group1_data)
                n2 = len(group2_data)
                
                # Combine the data from both groups and compute ranks
                combined_data = np.concatenate([group1_data, group2_data])
                ranks = rankdata(combined_data)
                
                # Calculate average ranks for each group
                avg_rank1 = np.mean(ranks[:n1])
                avg_rank2 = np.mean(ranks[n1:])
                
                # Total number of observations in both groups
                N = n1 + n2
                # Standard error calculation
                SE = np.sqrt((N * (N + 1) / 12) * (1/n1 + 1/n2))
                
                # Compute the z-statistic
                z = (avg_rank1 - avg_rank2) / SE
                
                # Calculate the two-tailed p-value
                p_value = 2 * (1 - norm.cdf(np.abs(z)))
                p_values.append((groups[i], groups[j], p_value))
    
    # Apply Bonferroni correction
    n_tests = len(p_values)
    adjusted_p_values = [(g1, g2, min(p * n_tests, 1.0)) for g1, g2, p in p_values]
    return adjusted_p_values

In [ ]:
for col in numerical_columns:
    print(f"Post-hoc pairwise comparisons for {col}:")
    
    # Group data by quality_categorical for the current column
    groups_data = [df_wine_clean[df_wine_clean['quality_categorical'] == quality][col].values 
                   for quality in ['Bad', 'Medium', 'Good']]
    
    # Perform Dunn's test
    results = dunn_test(groups_data, ['Bad', 'Medium', 'Good'])
    
    # Print results in scientific notation
    for group1, group2, p_value in results:
        print(f"Comparison: {group1} vs {group2}, p-value: {p_value:.3e}")
        if p_value < 0.05:
            print(f"  Significant difference between {group1} and {group2}")
        else:
            print(f"  No significant difference between {group1} and {group2}")
    print("-" * 50)

## Section Highlights:

- The correlation values with the quality variable are low, suggesting that a linear regression model may not be the most suitable.
- The separation into 3 categories (Bad, Medium, Good) for the wines shows some distinction in the data, as seen in the pairwise scatterplots. However, it is important to note that the number of wines in the "Medium" category was much higher than in the other categories.
- A significant difference in the means of features across the 3 categories was observed through the Kruskal-Wallis test, with statistical significance at the 5% level.
- The variables pH and sulfates had the highest p-values (around 3%).
- The Dunn test, applied to each feature, indicated that in the majority of cases, there was a statistically significant difference between the 3 classes.
- All available features will be used for model development.

**Next steps:**
- Evaluate the impact of the 3 categories in a classification model versus a regression model.
- Analyze the impact of outliers on model performance.
- Train several models to compare their performances.
- Perform hyperparameter tuning on the models.
- Evaluate the final model on the test set.

In [ ]:
# Save the data for modelling:
import os

# Define file paths
train_path = os.path.join('..', 'data', 'train.csv')
valid_path = os.path.join('..', 'data', 'validation.csv')
test_path = os.path.join('..', 'data', 'test.csv')

# Save the datasets as CSV files
pd.concat([X_train, y_train], axis = 1).to_csv(train_path, index=False)
pd.concat([X_valid, y_valid], axis = 1).to_csv(valid_path, index=False)
pd.concat([X_test, y_test], axis = 1).to_csv(test_path, index=False)